In [1]:
# Dependencies


import sys

sys.path.append("../")

import numpy as np
import functools as ft
import networkx as nx
from graph.graph import process_dataset, transform
from hdc import hdv, bind, bundle, sbundle, ItemMemory, hdvw, hdva, cosim

In [2]:
def init_vertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [3]:
def init_node_mem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.add_vector(f"mem{n}", graph.nodes[n]["mem"])

In [4]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if cosim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if cosim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.add_vector(f"mem{n}_{i}", mem)

In [5]:
def init_graph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.add_vector("G", G)
    return G

In [6]:
def check_edge(G, A, B, threshold):
    return cosim(B, bind([G, A])) > threshold

In [7]:
def node_memory_reconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = node_memory_reconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [8]:
def graph_memory_reconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graph_memory_reconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [9]:
def shortest_path(G, A, B, xs, distance, threshold, memory, visited):
    label, value, sim = memory.cleanup(bind([G, A]))

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: check_edge(G, A, x, threshold), xs))

    dis, lab = min(
        list(
            map(
                lambda a: shortest_path(
                    G, a, B, xs, distance + 1, threshold, memory, visited
                ),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

In [10]:
# Constants


NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [11]:
def test_check_edge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = check_edge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [12]:
def test_node_memory_reconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = node_memory_reconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(cosim(graph.nodes[0]["mem"], memsi[0])),
        )

In [13]:
def test_retrain():
    mem = ItemMemory()

    g = nx.gnm_random_graph(NODES, EDGES)
    init_vertices(g, DIMENSIONS)
    init_node_mem(g, mem)
    retrain(g, mem, THRESHOLD)
    G = init_graph(g, mem)


test_retrain()

In [14]:
def test_shortest_path():
    mem = ItemMemory()

    g = nx.gnm_random_graph(NODES, EDGES)
    init_vertices(g, DIMENSIONS)
    init_node_mem(g, mem)
    G = init_graph(g, mem)

    hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
    origin = 0

    test_check_edge(G, g, THRESHOLD)

    for m in g.nodes():
        if origin == m:
            continue
        path = nx.shortest_path(g, origin, m)
        oracle = len(path) - 1
        visited = {}
        test = shortest_path(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem, visited)
        testPath = list(map(int, test[1].split("mem")[1:]))
        print(
            f"{origin}_{m:02} =>",
            oracle,
            test[0],
            "T  " if oracle == test[0] else "F X",
            path,
            testPath,
            nx.is_path(g, testPath),
        )

In [15]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    init_vertices(graph, DIMENSIONS)
    init_node_mem(graph, memory)
    graph = retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # test_node_memory_reconstruction(G, graph, 1)
    # test_check_edge(G, graph, THRESHOLD)


# main()
test_shortest_path()

0 0.00000


0_01 => 2 2 T   [0, 26, 1] [0, 26, 1] True
0_02 => 2 2 T   [0, 21, 2] [0, 15, 2] True
0_03 => 2 2 T   [0, 16, 3] [0, 11, 3] True
0_04 => 2 2 T   [0, 19, 4] [0, 14, 4] True
0_05 => 2 2 T   [0, 26, 5] [0, 8, 5] True
0_06 => 2 2 T   [0, 18, 6] [0, 15, 6] True
0_07 => 2 2 T   [0, 15, 7] [0, 15, 7] True
0_08 => 1 1 T   [0, 8] [0, 8] True
0_09 => 3 3 T   [0, 8, 24, 9] [0, 8, 5, 9] True
0_10 => 2 2 T   [0, 19, 10] [0, 11, 10] True
0_11 => 1 1 T   [0, 11] [0, 11] True
0_12 => 1 1 T   [0, 12] [0, 12] True
0_13 => 2 2 T   [0, 26, 13] [0, 15, 13] True
0_14 => 1 1 T   [0, 14] [0, 14] True
0_15 => 1 1 T   [0, 15] [0, 15] True
0_16 => 1 1 T   [0, 16] [0, 16] True
0_17 => 2 2 T   [0, 16, 17] [0, 16, 17] True
0_18 => 1 1 T   [0, 18] [0, 18] True
0_19 => 1 1 T   [0, 19] [0, 19] True
0_20 => 2 2 T   [0, 11, 20] [0, 8, 20] True
0_21 => 1 1 T   [0, 21] [0, 21] True
0_22 => 2 2 T   [0, 21, 22] [0, 18, 22] True
0_23 => 2 2 T   [0, 8, 23] [0, 8, 23] True
0_24 => 2 2 T   [0, 8, 24] [0, 8, 24] True
0_25 => 2 2